# Summary table

This notebook tabulates key aggregates for each country-sector, namely: total output, intermediate use, value added, intermediate sales, final sales, and exports.

In [42]:
import numpy as np
import pandas as pd
import os
import re
import duckdb

## Set up

In [43]:
inputfolder = 'ADB-MRIO'
version = None
# inputfolder = 'ADB-MRIO_jun2023'
# version = 'jun2023'

filelist = [file for file in os.listdir(f'../data/interim/{inputfolder}') if not file.startswith('.')]
filelist.sort()

In [44]:
sectors = pd.read_excel('../data/interim/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

G = 73      # Number of countries + ROW
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [45]:
def zeroout(matrix, row=None, col=None, inverse=False, G=G):
    '''
    Zeroes out the NxN block diagonals of a GNxGN matrix. If inverse=True, all elements but the NxN block
    diagonals are zeroed out. An arbitrary NxN block can be zeroed out by passing ind() in the row and col 
    arguments. 
    '''
    Nrow = matrix.shape[0] // G
    Ncol = matrix.shape[1] // G
    zeroed_matrix = np.copy(matrix)
    
    if row is not None and col is not None:
        zeroed_matrix[np.ix_(row, col)] = 0
    else:
        for k in range(G):
            zeroed_matrix[(k*Nrow):(k*Nrow)+Nrow, (k*Ncol):(k*Ncol)+Ncol] = 0
    
    if inverse:
        return matrix - zeroed_matrix
    else:
        return zeroed_matrix

## Compile table

In [46]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Zd = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))), inverse=True)
    Yd = zeroout(Y, inverse=True)
    y = np.sum(Y, axis=1)
    ez = np.sum(Zd, axis=1)
    ey = np.sum(Yd, axis=1)

    DFi = pd.DataFrame({
        't': year,
        's': np.arange(1, G+1).repeat(N),
        'i': np.tile(sectors['ind'], G),
        'i5': np.tile(sectors['ind5'], G),
        'i15': np.tile(sectors['ind15'], G),
        'x': x,
        'zuse': zuse,
        'va': va,
        'zsales': zsales,
        'y': y,
        'e': ez + ey,
        'ez': ez,
        'ey': ey
    })
    DF = pd.concat([DF, DFi], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = 'summary.csv'
else:
    outputfilename = f'summary_{version}.csv'

DF.to_csv(f'../data/final/{outputfilename}', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
